# Rainband Data

This notebook is used to process the shear dataset and perform subsequent processing needed for rainband lightning burst analysis. We define rainband lightning as lightning occurring within 200-400km of storm center.

This notebook includes the following sections:
- Combining Shear Data Files
- Calculating Shear Quadrant
- Binning the Lightning Data
- Joining the Datasets
- Calculating Current Category and Intensification Change

This notebook should be executed after the `data_processing.ipynb` notebook. 

In the Combining Shear Data Files section, we use [intermediate_data/Filtered_Reduced_Trackfile.csv](intermediate_data/Filtered_Reduced_Trackfile.csv) to get the list of TCs to include in our analysis. We then look for `.mat` files that contain the shear angle and magnitude data for these TCs and create `intermediate_data/unbinned_shear_data.txt` with all the lightning events and their shear angles/magnitudes.

In the Calculating Shear Quadrant section, we use the shear angle to calculate the shear quadrant that each lightning event belongs to. We first calculate the direct distance of the lightning to the storm center using the hypothenuse method to filter down the dataset to lightning belonging to the rainband only (200-400km of storm center). We use the following formula to get the shear angle: 
```
shearAngleCG = geoAngleCG - shearAngle
```
We calculate geoAngleCG (geographic angle of lightning relative to north, where lightning due east of the storm center would give an angle of 90 degrees) using arctan and the distance east/north of the storm center. shearAngle is provided in the `.mat` dataset and included in the `unbinned_shear_data.txt` file. We then assign the lightning event to a shear quadrant using the following graphic (note that the graphic shows the shear vector to the right, we calculate angle relative to shear vector going north):

<img src="../shear_quadrant_graphic.png" width="200" height="200">

Based off shearAngleCG:
- 0-90 - Downshear Left (DL)
- 90-180 - Downshear Right (DR)
- 180-270 - Upshear Right (UR)
- 270 -360 - Upshear Left (UL)

The Binning the Lightning Data section creates 30 minute bins for use in evaluating lightning burst thresholds and comparison with TC wind and pressure data. We use the `intermediate_data/storm_time_period.csv` file created in the `data_processing.ipynb` notebook to ensure that the entirety of each storm is retained regardless of if rainband lightning is detected for the whole storm length. We group by shear quadrant as well as time bin and storm code.

In Joining the Datasets, we join the binned lightning dataset with the trackfile dataset. We join each 30-minute bin with the nearest wind and pressure data. This section does not create any files.

The last part of this notebook, Calculating Current Category and Intensification Change, calculates category and intensification change bins for use in the burst threshold analysis. This portion will create the `data/rainband_shear_timebin_joined.csv` file for use in analysis.

This notebook outputs the following files:
- `intermediate_data/filtered_tc_list.csv`
- `intermediate_data/unbinned_shear_data.txt`
- `data/rainband_shear_timebin_joined.csv`*

*denotes files used in the subsequent lightning burst threshold analysis.

## Requirements
This notebook requires the following libraries not built-in to Python:
- pandas
- polars
- scipy
- numpy

This notebook requires the following files created in [data_processing.ipynb](data_processing.ipynb)
- `intermediate_data/Filtered_Reduced_Trackfile.csv`
- `intermediate_data/storm_time_period.csv`

We assume that the shear data is provided in `.mat` files containing `_Intensity_Shear.mat` in the file name. We also assume that the raw data is provided in the following folder structure:
```
.
├── year
|   └── basin
|       └── storm number
|           └── shear file
```
example:
```
.
├── 10
|   └── ATL
|       └── 1
|           └── ATL_10_1_Intensity_Shear.mat
```

### Combining Shear Data Files
We start by combining the individual shear data files.

In [2]:
import os
import glob
import pandas as pd
import polars as pl
from scipy.io import loadmat
import numpy as np

Import the filtered trackfile data to extract a list of unique storm codes for storms category 1 or higher. We use this to limit the shear files we process to streamline the process and reduce the workload.

In [3]:
# Import the filtered trackfile for the list of storm codes in analysis
trackfile = pd.read_csv("intermediate_data/Filtered_Reduced_Trackfile.csv")
tc_list = trackfile[["storm_code", "storm_name"]].drop_duplicates()
tc_list.reset_index(drop=True, inplace=True)
tc_list.head()

,storm_code,storm_name
0,ATL_20_28,Zeta
1,ATL_20_28,Twenty-Eig
2,SHEM_20_4,Sarai
3,SHEM_16_7,Victor
4,ATL_17_8,Gert


In [ ]:
# Export filtered TC list for later use
tc_list.to_csv("intermediate_data/filtered_tc_list.csv", index=False)

Restart the kernel here and run the imports again to free up memory. Import the TC list created earlier to use as a filter in grabbing shear data files.

In [ ]:
# Restart kernel and import unique value csv
tc_list = pd.read_csv("intermediate_data/filtered_tc_list.csv")

Using the storm codes, we look for each TC's shear file in the base path (refers to thumb drive). If the TC has a shear file, we append it to the dataframe. We drop rows with null shear values.

In [5]:
# Define base path and list of storm codes
base_path = "/mnt/d/WWLLN_TC_Data_2010_2020/"
storm_codes = tc_list["storm_code"]

In [ ]:
# List to store data efficiently
data_list = []
processed_storms = []

# Process each storm code
for storm_code in storm_codes:
    if storm_code in processed_storms:
        continue
    basin, year, number = storm_code.split("_")
    folder_path = os.path.join(base_path, year, basin, number)
    file_pattern = os.path.join(folder_path, "*_Intensity_Shear.mat")
    files = glob.glob(file_pattern)

    if files:
        filename = files[0]  # Pick the first matching file
        mat_data = loadmat(filename)

        # Print keys to inspect the structure
        print(f"Loaded file: {filename}")

        # Create a DataFrame
        if 'cg_IntenShear' in mat_data:
            data_array = mat_data['cg_IntenShear']
            # Convert NumPy array to a Polars DataFrame directly
            df_temp = pd.DataFrame(data_array, columns=['year', 'month', 'day', 'hour', 'min', 'second', 'lat',
              'long', 'distance_east', 'distance_north', 'category', 'intensity_change',
              'trackfile_id', 'shear_magnitude', 'shear_angle'])
            # Drop NaN (null) rows before appending
            df_temp = df_temp.dropna()
            # Add a new column for storm_code
            df_temp["storm_code"] = storm_code
            data_list.append(df_temp)
        else:
            print("Variable not found in .mat file")
    else:
        print(f"{storm_code} does not have a shear file.")

# Efficiently concatenate all Polars DataFrames
shear_data = pd.concat(data_list, ignore_index=True)

# printed output log saved in separate txt file for reference

In [ ]:
# Display the final DataFrame
shear_data.head()

,year,month,day,hour,min,second,lat,long,distance_east,distance_north,category,intensity_change,trackfile_id,shear_magnitude,shear_angle,storm_code
0,2016.0,1.0,14.0,3.0,0.0,6.4994,-7.2386,-170.1965,-591.885,623.735,0.0,1.0,121.0,221.0,147.0,SHEM_16_7
1,2016.0,1.0,14.0,3.0,0.0,6.6831,-7.1552,-170.2882,-602.111,633.008,0.0,1.0,121.0,221.0,147.0,SHEM_16_7
2,2016.0,1.0,14.0,3.0,0.0,6.8064,-7.2494,-170.2057,-592.885,622.534,0.0,1.0,121.0,221.0,147.0,SHEM_16_7
3,2016.0,1.0,14.0,3.0,0.0,6.9387,-7.2733,-170.1481,-586.501,619.876,0.0,1.0,121.0,221.0,147.0,SHEM_16_7
4,2016.0,1.0,14.0,3.0,0.0,6.7190,-7.2224,-170.1888,-591.057,625.536,0.0,1.0,121.0,221.0,147.0,SHEM_16_7


We export the unbinned shear data for future reference.

In [ ]:
# Export the unbinned shear data as a tab-separated txt file
shear_data.to_csv("intermediate_data/unbinned_shear_data.txt", sep="\t", index=False)

### Calculating Shear Quadrant
We can restart the kernel here and read in the data again to free up space.

In [ ]:
# Import unbinned shear data file
shear_data = pd.read_csv("intermediate_data/unbinned_shear_data.txt", delimiter="\t")

In [9]:
# Assign column names to the data
shear_data[['year', 'month', 'day', 'hour','min','category','intensity_change','trackfile_id','shear_magnitude','shear_angle']] = shear_data[['year', 'month', 'day', 'hour','min','category','intensity_change','trackfile_id','shear_magnitude','shear_angle']].astype(int)
shear_data.head()

,year,month,day,hour,min,second,lat,long,distance_east,distance_north,category,intensity_change,trackfile_id,shear_magnitude,shear_angle,storm_code
0,2016,1,14,3,0,6.4994,-7.2386,-170.1965,-591.885,623.735,0,1,121,221,147,SHEM_16_7
1,2016,1,14,3,0,6.6831,-7.1552,-170.2882,-602.111,633.008,0,1,121,221,147,SHEM_16_7
2,2016,1,14,3,0,6.8064,-7.2494,-170.2057,-592.885,622.534,0,1,121,221,147,SHEM_16_7
3,2016,1,14,3,0,6.9387,-7.2733,-170.1481,-586.501,619.876,0,1,121,221,147,SHEM_16_7
4,2016,1,14,3,0,6.7190,-7.2224,-170.1888,-591.057,625.536,0,1,121,221,147,SHEM_16_7


Calculate the direct distance to storm center of each lightning event. Since the distance to center is generally close enough, we use a simplified hypotenuse calculation method instead of the Great Circle method.

In [10]:
# Calculate distance center using Pythagorean theorem
shear_data['distance_center'] = np.sqrt(shear_data['distance_east'] ** 2 + shear_data['distance_north'] ** 2)

Filter the data to rainband only. We define rainband as 200-400km of storm center.

In [11]:
# Create a rainband indicator, rainband is between 200 and 400 km of storm center
shear_data["rainband_ind"] = shear_data["distance_center"].apply(
    lambda x: 1 if (x >= 200 and x <= 400) else 0
)

# Filter the dataset to where rainband indicator = 1
rainband_data = shear_data[shear_data['rainband_ind'] == 1].copy()

Calculate shear angle for each lightning event using the following formula:

```
shearAngleCG = geoAngleCG - shearAngle
```
geoAngleCG is geographic angle of lightning to storm center relative to N = 0 degrees, where lightning due east of storm center would yield 90 degrees

shearAngle is provided in the shear dataset

We create a function to calculate the geographic angle relative to north. Apply it to the dataframe and then use the calculated geographic angle to get the shearAngleCG part of the equation.

In [12]:
def get_lightning_angle(north_south, east_west):
    angle_rad = np.arctan2(east_west, north_south)  # atan2(y, x)
    angle_deg = np.degrees(angle_rad)  # Convert radians to degrees
    return angle_deg % 360  # Ensure the angle is in [0, 360] range

In [13]:
# Apply function to data
rainband_data["geo_angle"] = get_lightning_angle(rainband_data["distance_north"], rainband_data["distance_east"])
rainband_data.head()

,year,month,day,hour,min,second,lat,long,distance_east,distance_north,category,intensity_change,trackfile_id,shear_magnitude,shear_angle,storm_code,distance_center,rainband_ind,geo_angle
25,2016,1,14,3,1,34.2911,-10.1733,-165.2575,-46.7057,297.411,0,1,121,221,147,SHEM_16_7,301.056017,1,351.075109
59,2016,1,14,3,3,32.6359,-10.4485,-166.4983,-182.3480,266.810,0,1,121,221,147,SHEM_16_7,323.169258,1,325.649828
60,2016,1,14,3,3,32.7588,-10.4338,-166.6904,-203.3630,268.445,0,1,121,221,147,SHEM_16_7,336.777713,1,322.853913
61,2016,1,14,3,3,32.6358,-10.4354,-166.7021,-204.6420,268.267,0,1,121,221,147,SHEM_16_7,337.410035,1,322.662541
62,2016,1,14,3,3,32.6992,-10.4359,-166.6930,-203.6460,268.211,0,1,121,221,147,SHEM_16_7,336.762281,1,322.791493


In [14]:
# Calculate shearAngleCG by subtracting the angles, make sure output is positive
rainband_data["shearAngleCG"] = (rainband_data["geo_angle"] - rainband_data["shear_angle"]) % 360
rainband_data.head()

,year,month,day,hour,min,second,lat,long,distance_east,distance_north,category,intensity_change,trackfile_id,shear_magnitude,shear_angle,storm_code,distance_center,rainband_ind,geo_angle,shearAngleCG
25,2016,1,14,3,1,34.2911,-10.1733,-165.2575,-46.7057,297.411,0,1,121,221,147,SHEM_16_7,301.056017,1,351.075109,204.075109
59,2016,1,14,3,3,32.6359,-10.4485,-166.4983,-182.3480,266.810,0,1,121,221,147,SHEM_16_7,323.169258,1,325.649828,178.649828
60,2016,1,14,3,3,32.7588,-10.4338,-166.6904,-203.3630,268.445,0,1,121,221,147,SHEM_16_7,336.777713,1,322.853913,175.853913
61,2016,1,14,3,3,32.6358,-10.4354,-166.7021,-204.6420,268.267,0,1,121,221,147,SHEM_16_7,337.410035,1,322.662541,175.662541
62,2016,1,14,3,3,32.6992,-10.4359,-166.6930,-203.6460,268.211,0,1,121,221,147,SHEM_16_7,336.762281,1,322.791493,175.791493


Next, let's calculate the distance X and Y of each lightning event relative to center, rotated on the shear angle.

We use the following formulas:
```
distanceY=dist_center.*sind(shearAngleCG)
distanceX=dist_center.*cosd(shearAngleCG)
```

dist_center is distance in km of lightning from storm center (direct distance), we can use the hypothenuse we calculated earlier.

We start by defining a sind and cosd function since Python's built-in sin function only accepts input in radians (not degrees).

In [15]:
def sind(angle_degrees):
    return np.sin(np.radians(angle_degrees))

In [16]:
def cosd(angle_degrees):
    return np.cos(np.radians(angle_degrees))

Next, let's use these functions to calculate the distanceX and distanceY.

In [17]:
# Calculate distanceX and distanceY using formula
rainband_data["distanceX"] = rainband_data["distance_center"]*sind(rainband_data["shearAngleCG"])
rainband_data["distanceY"] = rainband_data["distance_center"]*cosd(rainband_data["shearAngleCG"])
rainband_data.head()

,year,month,day,hour,min,second,lat,long,distance_east,distance_north,...,trackfile_id,shear_magnitude,shear_angle,storm_code,distance_center,rainband_ind,geo_angle,shearAngleCG,distanceX,distanceY
25,2016,1,14,3,1,34.2911,-10.1733,-165.2575,-46.7057,297.411,...,121,221,147,SHEM_16_7,301.056017,1,351.075109,204.075109,-122.810944,-274.867600
59,2016,1,14,3,3,32.6359,-10.4485,-166.4983,-182.3480,266.810,...,121,221,147,SHEM_16_7,323.169258,1,325.649828,178.649828,7.614760,-323.079533
60,2016,1,14,3,3,32.7588,-10.4338,-166.6904,-203.3630,268.445,...,121,221,147,SHEM_16_7,336.777713,1,322.853913,175.853913,24.348937,-335.896349
61,2016,1,14,3,3,32.6358,-10.4354,-166.7021,-204.6420,268.267,...,121,221,147,SHEM_16_7,337.410035,1,322.662541,175.662541,25.518542,-336.443659
62,2016,1,14,3,3,32.6992,-10.4359,-166.6930,-203.6460,268.211,...,121,221,147,SHEM_16_7,336.762281,1,322.791493,175.791493,24.713726,-335.854233


Let's assign each lightning event to a shear quadrant using the shear angle calculated earlier.

Based off shearAngleCG:
- 0-90 - Downshear Left (DL)
- 90-180 - Downshear Right (DR)
- 180-270 - Upshear Right (UR)
- 270 -360 - Upshear Left (UL)

(note that the graphic shows the shear vector to the right, we calculate angle relative to shear vector going north):

<img src="../shear_quadrant_graphic.png" width="200" height="200">

In [ ]:
# Define conditions and choices for the 4 quadrants
conditions = [
    (rainband_data["shearAngleCG"] >= 0) & (rainband_data["shearAngleCG"] < 90),
    (rainband_data["shearAngleCG"] >= 90) & (rainband_data["shearAngleCG"] < 180),
    (rainband_data["shearAngleCG"] >= 180) & (rainband_data["shearAngleCG"] < 270),
    (rainband_data["shearAngleCG"] >= 270) & (rainband_data["shearAngleCG"] < 360),
]
choices = ["DR", "UR", "UL", "DL"]

# Apply to create new column shear_quad
rainband_data["shear_quad"] = np.select(conditions, choices, default="Unknown")
rainband_data.head()

,year,month,day,hour,min,second,lat,long,distance_east,distance_north,...,shear_magnitude,shear_angle,storm_code,distance_center,rainband_ind,geo_angle,shearAngleCG,distanceX,distanceY,shear_quad
25,2016,1,14,3,1,34.2911,-10.1733,-165.2575,-46.7057,297.411,...,221,147,SHEM_16_7,301.056017,1,351.075109,204.075109,-122.810944,-274.867600,UL
59,2016,1,14,3,3,32.6359,-10.4485,-166.4983,-182.3480,266.810,...,221,147,SHEM_16_7,323.169258,1,325.649828,178.649828,7.614760,-323.079533,UR
60,2016,1,14,3,3,32.7588,-10.4338,-166.6904,-203.3630,268.445,...,221,147,SHEM_16_7,336.777713,1,322.853913,175.853913,24.348937,-335.896349,UR
61,2016,1,14,3,3,32.6358,-10.4354,-166.7021,-204.6420,268.267,...,221,147,SHEM_16_7,337.410035,1,322.662541,175.662541,25.518542,-336.443659,UR
62,2016,1,14,3,3,32.6992,-10.4359,-166.6930,-203.6460,268.211,...,221,147,SHEM_16_7,336.762281,1,322.791493,175.791493,24.713726,-335.854233,UR


### Binning the Lightning Data

Next, we'll create 30-minute timebins sectored by the shear quadrants. This means each TC will have 4 rows for the same timebin, one for each shear quadrant. We start by importing the min and max timestamps for each storm calculated using the trackfile. We do this to ensure we retain the entire length of each storm, regardless of if there is WWLLN lightning between 200-400km during the whole length of the storm.

In [ ]:
# Import storm time period data
full_time_period_df = pd.read_csv('intermediate_data/storm_time_period.csv')

In [ ]:
# Ensure sec column is valid
rainband_data['sec'] = rainband_data['second'].apply(lambda x: 0 if x == 60 else x)

# Create a datetime column
rainband_data['datetime'] = pd.to_datetime(
    rainband_data['year'].astype(str) + '-' +
    rainband_data['month'].astype(str).str.zfill(2) + '-' +
    rainband_data['day'].astype(str).str.zfill(2) + ' ' +
    rainband_data['hour'].astype(str).str.zfill(2) + ':' +
    rainband_data['min'].astype(str).str.zfill(2) + ':' +
    rainband_data['sec'].astype(str).str.zfill(2)
)

# Define a function to apply the 30-minute binning for each storm_code group
def add_time_bin(group):
    group['time_bin'] = group['datetime'].dt.floor('30min')
    return group

# Group by storm_code and apply the binning function
rainband_data = rainband_data.groupby('storm_code', group_keys=False).apply(add_time_bin)

# Group by bins and get the count per 30-minute bin
rainband_data_grouped = rainband_data.groupby(['storm_code', 'time_bin', 'shear_quad']).size().reset_index(name='lightning_count')

# Define all shear_quad options
shear_quad_options = ["DR", "UR", "UL", "DL"]

# Function to ensure all time bins have all shear_quad values
def add_missing_bins(group):
    storm_code = group['storm_code'].iloc[0]
    # Use track file start and end dates to retain entirety of storm length
    min_time = full_time_period_df.loc[full_time_period_df['storm_code'] == storm_code, 'min_time'].values[0]
    max_time = full_time_period_df.loc[full_time_period_df['storm_code'] == storm_code, 'max_time'].values[0]

    # Create a full range of 30-minute bins for the time period of this storm
    full_bins = pd.DataFrame({'time_bin': pd.date_range(min_time, max_time, freq='30min')})

    # Create all possible combinations of (time_bin, shear_quad)
    shear_quad_expanded = full_bins.assign(key=1).merge(
        pd.DataFrame({'shear_quad': shear_quad_options, 'key': 1}), on='key'
    ).drop(columns=['key'])

    # Merge with original data
    merged = shear_quad_expanded.merge(group[['storm_code', 'time_bin', 'shear_quad', 'lightning_count']],
                                       how='left', on=['time_bin', 'shear_quad'])

    # Fill missing storm_code with the first valid entry in the group
    merged['storm_code'] = group['storm_code'].iloc[0]

    # Fill missing lightning_count with 0
    merged['lightning_count'] = merged['lightning_count'].fillna(0).astype(int)

    return merged

# Apply the function to ensure all (time_bin, shear_quad) combinations exist
rainband_data_timebin = rainband_data_grouped.groupby('storm_code', group_keys=False).apply(add_missing_bins)

# Sort the final result
rainband_data_timebin = rainband_data_timebin.sort_values(by=['storm_code', 'time_bin', 'shear_quad'])

# Print the resulting DataFrame with the new 'time_bin' and 'lightning_count' columns
rainband_data_timebin.head()

### Joining the Datasets
We join the binned lightning data with the trackfile data to get the closest wind and pressure data. Start by reading in the trackfile data and converting the data types.

In [ ]:
# Import trackfile
reduced_track_file = pd.read_csv("intermediate_data/Filtered_Reduced_Trackfile.csv")
reduced_track_file.head()

,year,month,day,hour,lat,lon,pressure,knots,storm_code,storm_name,category,basin
0,2020,10,20,0,12.1,-80.0,0,15,ATL_20_28,Zeta,2,ATL
1,2020,10,20,6,12.5,-80.1,0,15,ATL_20_28,Zeta,2,ATL
2,2020,10,20,12,12.8,-80.2,0,15,ATL_20_28,Zeta,2,ATL
3,2020,10,20,18,13.2,-80.3,0,15,ATL_20_28,Zeta,2,ATL
4,2020,10,21,0,13.8,-80.4,0,15,ATL_20_28,Zeta,2,ATL


In [23]:
# Convert the datetime columns to the correct data type
rainband_data_timebin["time_bin"] = pd.to_datetime(rainband_data_timebin["time_bin"])

rainband_data_timebin["year"] = rainband_data_timebin["time_bin"].dt.year.astype('int64')
rainband_data_timebin["month"] = rainband_data_timebin["time_bin"].dt.month.astype('int64')
rainband_data_timebin["day"] = rainband_data_timebin["time_bin"].dt.day.astype('int64')
rainband_data_timebin["hour"] = rainband_data_timebin["time_bin"].dt.hour.astype('int64')
rainband_data_timebin["minute"] = rainband_data_timebin["time_bin"].dt.minute.astype('int64')

In [ ]:
# Convert to polars dataframe for more efficient processing
rainband_data_timebin_pl = pl.from_pandas(rainband_data_timebin)
reduced_track_file_pl = pl.from_pandas(reduced_track_file)

Next, we join the rainband track data with the binned lightning data using the [`join_asof`](https://docs.pola.rs/api/python/stable/reference/dataframe/api/polars.DataFrame.join_asof.html) method. We look for the nearest track data to the lightning timebin.

In [ ]:
# Apply join_asof to lightning and track data
rainband_data_timebin_joined = rainband_data_timebin_pl.join_asof(
    reduced_track_file_pl,
    on="hour",
    by=["year", "month", "day","storm_code"],
    strategy="nearest",
    tolerance=24
)

In [ ]:
# Convert back to pandas dataframe
rainband_data_timebin_joined = rainband_data_timebin_joined.to_pandas()

In [ ]:
# Check dataframe sample
rainband_data_timebin_joined.head(20)

,time_bin,shear_quad,storm_code,lightning_count,year,month,day,hour,minute,lat,lon,pressure,knots,storm_name,category,basin
0,2010-06-20 18:00:00,DL,ATL_10_1,0,2010,6,20,18,0,12.0,-64.3,1011,15,Alex,2,ATL
1,2010-06-20 18:00:00,DR,ATL_10_1,0,2010,6,20,18,0,12.0,-64.3,1011,15,Alex,2,ATL
2,2010-06-20 18:00:00,UL,ATL_10_1,0,2010,6,20,18,0,12.0,-64.3,1011,15,Alex,2,ATL
3,2010-06-20 18:00:00,UR,ATL_10_1,0,2010,6,20,18,0,12.0,-64.3,1011,15,Alex,2,ATL
4,2010-06-20 18:30:00,DL,ATL_10_1,0,2010,6,20,18,30,12.0,-64.3,1011,15,Alex,2,ATL
5,2010-06-20 18:30:00,DR,ATL_10_1,0,2010,6,20,18,30,12.0,-64.3,1011,15,Alex,2,ATL
6,2010-06-20 18:30:00,UL,ATL_10_1,0,2010,6,20,18,30,12.0,-64.3,1011,15,Alex,2,ATL
7,2010-06-20 18:30:00,UR,ATL_10_1,0,2010,6,20,18,30,12.0,-64.3,1011,15,Alex,2,ATL
8,2010-06-20 19:00:00,DL,ATL_10_1,0,2010,6,20,19,0,12.0,-64.3,1011,15,Alex,2,ATL
9,2010-06-20 19:00:00,DR,ATL_10_1,0,2010,6,20,19,0,12.0,-64.3,1011,15,Alex,2,ATL


In [ ]:
# Check for any null values
rainband_data_timebin_joined.isnull().sum()

time_bin           0
shear_quad         0
storm_code         0
lightning_count    0
year               0
month              0
day                0
hour               0
minute             0
lat                0
lon                0
pressure           0
knots              0
storm_name         0
category           0
basin              0
dtype: int64

### Calculating Current Category and Intensification Change
After joining the WWLLN lightning events to its nearest wind and pressure data, we will then calculate the intensity change and category at each time bin. Intensity change is defined as the difference between the current wind speed and the wind speed 24 hours later. If the exact 24 hour timestamp cannot be found, we will compare with the closest timestamp to 24 hours. The current category of the TC at the time of the lightning event is evaluated based on the current wind speed. Once again, we refer to the [Saffir-Simpson Hurricane Wind Scale](https://www.nhc.noaa.gov/aboutsshws.php).

**Intensification Change Bins**
| Intensification Stage | Change in Winds (Knots) in 24 Hours (Jiang and Ramirez, 2013)|
| --------------------- | ----------------------|
| Weakening | <-30 to -10 |
| Neutral | -10 to 10 |
| Intensifying | 10 to >30 |

**TC Category Bins**
| TC Category | Sustained Winds (knots) | 
|  ---------- | ------------|
| 1 | 64-82 kt |
| 2 | 83-95 kt |
| 3 | 96-112 kt |
| 4 | 113-136 kt |
| 5 | 137 kt or higher | 

Start by creating the functions to categorize by wind speed.

In [ ]:
# Calculate 24-hour intensity change (forward-looking)
def knot_category(row):
    if 64 <= row['knots'] < 83:
        return 1
    elif 83 <= row['knots'] < 96:
        return 2
    elif 96 <= row['knots'] < 113:
        return 3
    elif 113 <= row['knots'] < 136:
        return 4
    elif row['knots'] >= 137:
        return 5
    else:
        return 'Unidentified'
# Calculate intensification bin
def intensification(row):
    if row['24_hour_knots_diff'] < -30:
        return 'Rapidly Weakening'
    elif -30 <= row['24_hour_knots_diff'] < -10:
        return 'Weakening'
    elif -10 <= row['24_hour_knots_diff'] < 10:
        return 'Neutral'
    elif 10 <= row['24_hour_knots_diff'] < 30:
        return 'Intensifying'
    elif 30 <= row['24_hour_knots_diff']:
        return 'Rapidly Intensifying'
    else:
        return 'Unidentified'

Calculate forward-looking 24-hour differences for wind speed and apply the above functions to the binned rainband data.

In [ ]:
pd.set_option('display.max_rows', 500)

# Sort by 'storm_code' and 'time_bin'
rainband_data_timebin_joined = rainband_data_timebin_joined.sort_values(by=['storm_code', 'time_bin','shear_quad'])

# Calculate forward-looking 24 hour difference columns for wind speed and pressure
rainband_data_timebin_joined['24_hour_knots_diff'] = rainband_data_timebin_joined.groupby(['storm_code','shear_quad'])['knots'].shift(periods=-48) - rainband_data_timebin_joined['knots']
rainband_data_timebin_joined['24_hour_pressure_diff'] = rainband_data_timebin_joined.groupby(['storm_code','shear_quad'])['pressure'].shift(periods=-48) - rainband_data_timebin_joined['pressure']

In [ ]:
# Apply categorization functions
rainband_data_timebin_joined['TC_Category'] = rainband_data_timebin_joined.apply(knot_category, axis=1)
rainband_data_timebin_joined['Intensification_Category'] = rainband_data_timebin_joined.apply(intensification, axis=1)

In [ ]:
# Check the shape of the dataframe
rainband_data_timebin_joined.shape

(584416, 20)

Export this dataset to the `data/` directory for use in [`lightning_threshold_rainband.ipynb`](../lightning_burst_identification/lightning_threshold_rainband.ipynb).

In [ ]:
# Export dataframe
rainband_data_timebin_joined.to_csv("data/rainband_shear_timebin_joined.csv", index=False)